In [1]:
# Importing packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Loading CDC data
Data = pd.read_csv('./Data/Underlying Cause of Death, 1999-2020.csv')
#Removing District of Columbia - Missing values for policy and just Washington D.C., not very relevant for the study
Data = Data[Data['State Code'] != 11]
Data.head()

,State,State Code,Year,Year Code,Deaths,Population,Crude Rate
0,Alabama,1.0,1999.0,1999.0,372,4430141,8.4
1,Alabama,1.0,2000.0,2000.0,389,4447100,8.7
2,Alabama,1.0,2001.0,2001.0,333,4467634,7.5
3,Alabama,1.0,2002.0,2002.0,334,4480089,7.5
4,Alabama,1.0,2003.0,2003.0,350,4503491,7.8


In [3]:
# Loading policy data and droping duplicate columns
Policy = pd.read_csv('./Data/DATABASE_michig.csv')
Policy = Policy.loc[:,['state','year','waitingh','permith']]
Policy.rename({'state':'State','year':'Year'},inplace = True, axis=1)
Data.drop(['Year Code'],inplace=True,axis=1)

In [4]:
# Merging data sets and creating new variables
# https://www.statefirearmlaws.org/ <- website where data is publicly available
Data = Data.merge(Policy,how='left',left_on=['State','Year'], right_on=['State','Year'])
# Creating outcome variable
Data['rate_calculated'] = round((Data['Deaths']/Data['Population'])*100000,1)
Data['log_rate'] = np.log(Data['rate_calculated'])

In [5]:
#Defining a function which can analyze the dataset in terms of missing values and type of columns
def Analysis(data):
    print("Analysing")
    print(data.info())
    #Calculating the share of missing values
    missing = pd.concat([data.isnull().sum(), 100 * data.isnull().mean()], axis=1)
    missing.columns=['Count', '%']
    missing = missing.sort_values(by='Count', ascending=False)
    print(missing)
    df=missing.iloc[0:5,0]
Analysis(Data)

Analysing
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1151 entries, 0 to 1150
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   State            1150 non-null   object 
 1   State Code       1150 non-null   float64
 2   Year             1100 non-null   float64
 3   Deaths           1151 non-null   int64  
 4   Population       1151 non-null   int64  
 5   Crude Rate       1151 non-null   object 
 6   waitingh         1100 non-null   float64
 7   permith          1100 non-null   float64
 8   rate_calculated  1151 non-null   float64
 9   log_rate         1151 non-null   float64
dtypes: float64(6), int64(2), object(2)
memory usage: 98.9+ KB
None
                 Count         %
Year                51  4.430930
waitingh            51  4.430930
permith             51  4.430930
State                1  0.086881
State Code           1  0.086881
Deaths               0  0.000000
Population           0  0.000000

In [6]:
# Dropping overall SUM variables - i.e., the variables that show the average for all states in the sample.
Data = Data.dropna()

In [7]:
# Adding census region data, further merging and dropping further columns
Region = pd.read_csv('https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv')
Region.columns
Region.rename({'State Code':'scode_char'},inplace=True,axis=1)
Data = Data.merge(Region,how='left',left_on=['State'],right_on=['State'])
Data.drop(['Crude Rate'],inplace=True,axis=1)

In [8]:
# Demographic control variables - Source: Census B.
rural = pd.read_csv('./Data/perc_rural.csv')
Data = Data.merge(rural,how='left',left_on=['State'], right_on=['State'])
education = pd.read_csv('./Data/education.csv')
Data = Data.merge(education,how='left',left_on=['State'], right_on=['State'])
hslower = pd.read_csv('./Data/hs_lower.csv')
Data = Data.merge(hslower,how='left',left_on=['State'], right_on=['State']) 
income = pd.read_csv('./Data/median_income.csv')
Data = Data.merge(income,how='left',left_on=['State'], right_on=['State'])
race_sex = pd.read_csv('./Data/race_sex.csv')
Data = Data.merge(race_sex,how='left',left_on=['State'], right_on=['State'])

In [9]:
# Long gun proxy - Source: CDC
long_guns = pd.read_csv('./Data/firearm_withrifles.csv')
long_guns['rate_all_firearms'] = round((long_guns['Deaths']/long_guns['Population'])*100000,1)
long_guns.drop(['State Code','Crude Rate','Population','Deaths'],inplace=True,axis=1)
Data = Data.merge(long_guns,how='left',left_on=['State','Year'], right_on=['State','Year'])
Data['rlong_guns'] = Data.rate_all_firearms - Data.rate_calculated
Data['log_long'] = np.log(1+Data.rlong_guns)

In [10]:
# Adding overall suicide rate - Source: CDC
everything = pd.read_csv('./Data/everything.csv')
everything['rate_combined'] = round((everything['Deaths']/everything['Population'])*100000,1)
everything['log_everything'] = np.log(everything['Crude Rate'])
everything.drop(['State Code','Crude Rate','Population','Deaths','Year Code'],inplace=True,axis=1)
Data = Data.merge(everything,how='left',left_on=['State','Year'], right_on=['State','Year'])
Data['all_except_handguns'] = Data.rate_combined - Data.rate_calculated
Data['lall_except_handguns'] = np.log(Data.all_except_handguns)

In [11]:
# Firearm prevalence proxy
prevalance = pd.read_csv('./Data/prevalance_guns.csv')
prevalance.rename({'STATE':'State'},inplace=True,axis=1)
prevalance = prevalance.loc[prevalance.Year == 2004,['State','BRFSS']]
prevalance.head()
Data = Data.merge(prevalance,how='left',left_on=['State'], right_on=['State'])

In [12]:
# Background checks: FBI
hcheck = pd.read_csv('./Data/NICS_Firearm_Checks.csv')
hcheck['tot_handgun'] = hcheck.Permit + hcheck.Handgun
hcheck.drop(['Long Gun','*Other','**Multiple','Admin','Permit_recheck',],inplace = True,axis=1)
Data = Data.merge(hcheck,how='left',left_on=['State','Year'], right_on=['State','Year'])

In [13]:
Data.describe()

,State Code,Year,Deaths,Population,waitingh,permith,rate_calculated,log_rate,perc_rural,perc_hs,...,log_long,rate_combined,log_everything,all_except_handguns,lall_except_handguns,BRFSS,Permit,Handgun,Totals,tot_handgun
count,1100.00000,1100.000000,1100.000000,1.100000e+03,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,...,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1.100000e+03,1.100000e+03,1.100000e+03,1.100000e+03
mean,29.32000,2009.500000,336.820000,6.120655e+06,0.180000,0.270909,6.400273,1.726299,0.294219,29.952000,...,0.765357,14.154636,2.607031,7.754364,2.008143,0.396930,9.547532e+04,9.166218e+04,3.349433e+05,1.871375e+05
std,15.63073,6.347175,329.491883,6.779964e+06,0.384362,0.444631,2.989830,0.547765,0.151425,3.919089,...,0.354875,4.190922,0.295044,2.207364,0.284363,0.135507,3.094115e+05,1.220980e+05,5.215865e+05,3.562352e+05
min,1.00000,1999.000000,11.000000,4.917800e+05,0.000000,0.000000,1.000000,0.000000,0.053201,20.100000,...,0.000000,6.000000,1.791759,3.500000,1.252763,0.104969,0.000000e+00,0.000000e+00,5.343000e+03,2.285000e+03
25%,17.00000,2004.000000,103.000000,1.800205e+06,0.000000,0.000000,4.075000,1.404814,0.145886,27.800000,...,0.515472,11.200000,2.415914,6.000000,1.791759,0.343355,2.000000e+00,1.889725e+04,8.807050e+04,3.354950e+04
50%,29.50000,2009.500000,247.000000,4.348607e+06,0.000000,0.000000,6.400000,1.856298,0.287658,29.400000,...,0.693147,13.600000,2.610070,7.500000,2.014903,0.405373,1.524550e+04,4.995100e+04,1.888375e+05,8.292500e+04
75%,42.00000,2015.000000,445.000000,6.978598e+06,0.000000,1.000000,8.300000,2.116256,0.394934,32.400000,...,0.993252,16.500000,2.803360,9.100000,2.208274,0.480447,8.338825e+04,1.143565e+05,3.631330e+05,1.951995e+05
max,56.00000,2020.000000,2108.000000,3.955704e+07,1.000000,1.000000,20.100000,3.000720,0.664653,39.400000,...,1.960095,31.300000,3.443618,15.300000,2.727853,0.631548,4.655016e+06,1.042466e+06,7.455065e+06,4.767339e+06


In [14]:
# Creating delay variable 
Data['Delay'] = np.where((Data.waitingh==1) | (Data.permith==1),1,0)
Data.set_index(['State','Region','Year'],inplace=True)

In [15]:
Data.head()

State Code  Deaths  Population  waitingh  permith  \
State   Region Year                                                        
Alabama South  1999.0         1.0     372     4430141       1.0      0.0   
               2000.0         1.0     389     4447100       0.0      0.0   
               2001.0         1.0     333     4467634       0.0      0.0   
               2002.0         1.0     334     4480089       0.0      0.0   
               2003.0         1.0     350     4503491       0.0      0.0   

                       rate_calculated  log_rate scode_char  \
State   Region Year                                           
Alabama South  1999.0              8.4  2.128232         AL   
               2000.0              8.7  2.163323         AL   
               2001.0              7.5  2.014903         AL   
               2002.0              7.5  2.014903         AL   
               2003.0              7.8  2.054124         AL   

                                 Division  perc_rural  ...  rate_combined  \
State   Region Year                                    ...                  
Alabama South  1999.0  East South Central    0.449754  ...           12.5   
               2000.0  East South Central    0.449754  ...           13.1   
               2001.0  East South Central    0.449754  ...           11.5   
               2002.0  East South Central    0.449754  ...           11.5   
               2003.0  East South Central    0.449754  ...           11.5   

                       log_everything  all_except_handguns  \
State   Region Year                                          
Alabama South  1999.0        2.525729                  4.1   
               2000.0        2.572612                  4.4   
               2001.0        2.442347                  4.0   
               2002.0        2.442347                  4.0   
               2003.0        2.442347                  3.7   

                       lall_except_handguns     BRFSS  Permit  Handgun  \
State   Region Year                                                      
Alabama South  1999.0              1.410987  0.525519       0    94544   
               2000.0              1.481605  0.525519      12    81983   
               2001.0              1.386294  0.525519       0    83885   
               2002.0              1.386294  0.525519       0    65294   
               2003.0              1.308333  0.525519       0    67985   

                       Totals  tot_handgun  Delay  
State   Region Year                                
Alabama South  1999.0  246756        94544      1  
               2000.0  221911        81995      0  
               2001.0  230187        83885      0  
               2002.0  221008        65294      0  
               2003.0  225479        67985      0  

[5 rows x 29 columns]

In [16]:
# Creating new data set which contains control and treatment group for analysis
Variation = ['California','Connecticut','Hawaii','Illinois',
                 'Iowa','Maryland','Massachusetts','Minnesota','Michigan','Missouri',
                     'Wisconsin','North Carolina','New Jersey','New York','Nebraska','Rhode Island','Virginia']
Change = Data.reset_index()
Change = Change.loc[Change['State'].isin(Variation)]
# Creating new variables 
Change['treatment'] = 1 - Change['Delay']
Change['treatment_group'] = np.where((Change.State=='Michigan') | (Change.State=='Virginia') | (Change.State=='Wisconsin') | (Change.State=='Missouri'),1,0)
Change.reset_index(inplace=True)
Change.drop(['index'],inplace=True,axis=1)
Change.set_index(['State'],inplace=True)
Change.head()

,Region,Year,State Code,Deaths,Population,waitingh,permith,rate_calculated,log_rate,scode_char,...,all_except_handguns,lall_except_handguns,BRFSS,Permit,Handgun,Totals,tot_handgun,Delay,treatment,treatment_group
State,,,,,,,,,,,,,,,,,,,,,
California,West,1999.0,6.0,1268,33499204,1.0,1.0,3.8,1.335001,CA,...,5.4,1.686399,0.214034,101132,371893,883144,473025,1,0,0
California,West,2000.0,6.0,1266,33871648,1.0,1.0,3.7,1.308333,CA,...,5.0,1.609438,0.214034,163145,328615,794506,491760,1,0,0
California,West,2001.0,6.0,1190,34479458,1.0,1.0,3.5,1.252763,CA,...,4.7,1.547563,0.214034,236271,280044,854569,516315,1,0,0
California,West,2002.0,6.0,1200,34871843,1.0,1.0,3.4,1.223775,CA,...,5.8,1.757858,0.214034,191714,235121,684390,426835,1,0,0
California,West,2003.0,6.0,1271,35253159,1.0,1.0,3.6,1.280934,CA,...,6.0,1.791759,0.214034,156003,158811,524431,314814,1,0,0


In [17]:
#Creating baseyear variable - This variable is necessary to run the did package in R, i.e. the event study staggered treatment regression.
Change['baseyear'] = np.where(Change.treatment==1,Change.Year*Change.treatment,np.NaN)
Change['byear'] = Change.groupby(['State'])['baseyear'].first()
Change.byear = Change.byear.fillna(0)
Change.drop(['baseyear'],inplace=True,axis=1)

In [18]:
Change.head(375)

,Region,Year,State Code,Deaths,Population,waitingh,permith,rate_calculated,log_rate,scode_char,...,lall_except_handguns,BRFSS,Permit,Handgun,Totals,tot_handgun,Delay,treatment,treatment_group,byear
State,,,,,,,,,,,,,,,,,,,,,
California,West,1999.0,6.0,1268,33499204,1.0,1.0,3.8,1.335001,CA,...,1.686399,0.214034,101132,371893,883144,473025,1,0,0,0.0
California,West,2000.0,6.0,1266,33871648,1.0,1.0,3.7,1.308333,CA,...,1.609438,0.214034,163145,328615,794506,491760,1,0,0,0.0
California,West,2001.0,6.0,1190,34479458,1.0,1.0,3.5,1.252763,CA,...,1.547563,0.214034,236271,280044,854569,516315,1,0,0,0.0
California,West,2002.0,6.0,1200,34871843,1.0,1.0,3.4,1.223775,CA,...,1.757858,0.214034,191714,235121,684390,426835,1,0,0,0.0
California,West,2003.0,6.0,1271,35253159,1.0,1.0,3.6,1.280934,CA,...,1.791759,0.214034,156003,158811,524431,314814,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Midwest,2016.0,55.0,319,5778708,0.0,0.0,5.5,1.704748,WI,...,2.251292,0.402903,199431,186300,561819,385731,0,1,1,2015.0
Wisconsin,Midwest,2017.0,55.0,313,5795483,0.0,0.0,5.4,1.686399,WI,...,2.360854,0.402903,182395,171610,526523,354005,0,1,1,2015.0
Wisconsin,Midwest,2018.0,55.0,295,5813568,0.0,0.0,5.1,1.629241,WI,...,2.322388,0.402903,131991,155441,452520,287432,0,1,1,2015.0


In [19]:
Change.rename({'Year':'year','State Code':'scode','Deaths':'deaths','Population':'population',
                   'Delay':'delay','Region':'region','Division':'division'},inplace=True,axis=1)

In [20]:
Change.describe()

,year,scode,deaths,population,waitingh,permith,rate_calculated,log_rate,perc_rural,perc_hs,...,lall_except_handguns,BRFSS,Permit,Handgun,Totals,tot_handgun,delay,treatment,treatment_group,byear
count,374.000000,374.000000,374.000000,3.740000e+02,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,...,374.000000,374.000000,3.740000e+02,374.000000,3.740000e+02,3.740000e+02,374.000000,374.000000,374.000000,374.000000
mean,2009.500000,28.529412,310.264706,8.532353e+06,0.454545,0.778075,3.731551,1.196234,0.210709,29.000000,...,1.944144,0.290381,1.078883e+05,77663.655080,3.418932e+05,1.855519e+05,0.879679,0.120321,0.235294,472.882353
std,6.352787,13.182283,303.371541,8.422058e+06,0.498597,0.416098,1.825679,0.500368,0.119905,3.503089,...,0.223379,0.131763,1.653069e+05,108146.193504,5.659219e+05,2.380712e+05,0.325772,0.325772,0.424751,853.645503
min,1999.000000,6.000000,11.000000,1.040402e+06,0.000000,0.000000,1.000000,0.000000,0.053201,20.100000,...,1.435085,0.104969,0.000000e+00,0.000000,5.343000e+03,4.010000e+03,0.000000,0.000000,0.000000,0.000000
25%,2004.000000,19.000000,98.000000,3.498810e+06,0.000000,1.000000,2.100000,0.741937,0.099496,27.700000,...,1.762132,0.185762,7.575500e+03,5003.000000,8.960175e+04,4.328975e+04,1.000000,0.000000,0.000000,0.000000
50%,2009.500000,27.000000,208.000000,6.043444e+06,0.000000,1.000000,3.500000,1.252763,0.144313,28.500000,...,1.945910,0.238227,4.766100e+04,37906.000000,1.992425e+05,9.565950e+04,1.000000,0.000000,0.000000,0.000000
75%,2015.000000,36.000000,413.000000,9.881567e+06,1.000000,1.000000,5.000000,1.609438,0.317269,31.300000,...,2.113225,0.407843,1.331715e+05,102049.750000,4.336172e+05,2.385838e+05,1.000000,0.000000,0.000000,0.000000
max,2020.000000,55.000000,1432.000000,3.955704e+07,1.000000,1.000000,10.400000,2.341806,0.409545,36.100000,...,2.624669,0.493786,1.036981e+06,714441.000000,7.455065e+06,1.597336e+06,1.000000,1.000000,1.000000,2015.000000


In [21]:
Change.drop(['scode_char','division','deaths'],inplace=True,axis=1)

In [22]:
# Creating variable for event-time study
Change['stimetotreatment'] = np.where(Change.byear !=0,Change.year - Change.byear, 0)

In [23]:
Change.head()

,region,year,scode,population,waitingh,permith,rate_calculated,log_rate,perc_rural,perc_hs,...,BRFSS,Permit,Handgun,Totals,tot_handgun,delay,treatment,treatment_group,byear,stimetotreatment
State,,,,,,,,,,,,,,,,,,,,,
California,West,1999.0,6.0,33499204,1.0,1.0,3.8,1.335001,0.067542,20.1,...,0.214034,101132,371893,883144,473025,1,0,0,0.0,0.0
California,West,2000.0,6.0,33871648,1.0,1.0,3.7,1.308333,0.067542,20.1,...,0.214034,163145,328615,794506,491760,1,0,0,0.0,0.0
California,West,2001.0,6.0,34479458,1.0,1.0,3.5,1.252763,0.067542,20.1,...,0.214034,236271,280044,854569,516315,1,0,0,0.0,0.0
California,West,2002.0,6.0,34871843,1.0,1.0,3.4,1.223775,0.067542,20.1,...,0.214034,191714,235121,684390,426835,1,0,0,0.0,0.0
California,West,2003.0,6.0,35253159,1.0,1.0,3.6,1.280934,0.067542,20.1,...,0.214034,156003,158811,524431,314814,1,0,0,0.0,0.0


In [24]:
#Downloading data set
Change.to_csv('./Data/ForR_controls.csv')
Data.to_csv('./Data/Check.csv')

In [25]:
Change.head()

,region,year,scode,population,waitingh,permith,rate_calculated,log_rate,perc_rural,perc_hs,...,BRFSS,Permit,Handgun,Totals,tot_handgun,delay,treatment,treatment_group,byear,stimetotreatment
State,,,,,,,,,,,,,,,,,,,,,
California,West,1999.0,6.0,33499204,1.0,1.0,3.8,1.335001,0.067542,20.1,...,0.214034,101132,371893,883144,473025,1,0,0,0.0,0.0
California,West,2000.0,6.0,33871648,1.0,1.0,3.7,1.308333,0.067542,20.1,...,0.214034,163145,328615,794506,491760,1,0,0,0.0,0.0
California,West,2001.0,6.0,34479458,1.0,1.0,3.5,1.252763,0.067542,20.1,...,0.214034,236271,280044,854569,516315,1,0,0,0.0,0.0
California,West,2002.0,6.0,34871843,1.0,1.0,3.4,1.223775,0.067542,20.1,...,0.214034,191714,235121,684390,426835,1,0,0,0.0,0.0
California,West,2003.0,6.0,35253159,1.0,1.0,3.6,1.280934,0.067542,20.1,...,0.214034,156003,158811,524431,314814,1,0,0,0.0,0.0


In [26]:
# Creating function for treatment effect calculations
import math
def calculation(a):
    return (math.exp(a) - 1) * 100

In [27]:
perc_suicide = calculation(0.228)
perc_handgun = calculation(0.399)
elasticity = perc_suicide / perc_handgun

In [28]:
print(elasticity)

0.5222675250366322
